Write a code using the DCGAN architecture that is able to generate images similar to handwritten digits in the MNIST dataset format. Considering that this architecture uses CNN networks, include the following items in the report:

(a) The topology of the generator and discriminator layers in tensor form \
(b) The way the loss functions and activation functions work (along with the reason for using them) \
(c) Batch normalization (how it works and its importance) \
(d) The need for a dropout layer \ 
(e) The method of generating noise \
(f) A complete explanation of how the ADAM optimizer works \
(g) Loss and accuracy plots for both the generator and the discriminator \
(h) Sample outputs of the generator network at several specific epochs, along with justification of the improvement in the network’s performance over time \

In [ ]:
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import *
from keras.optimizers import Adam
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

X_train = X_train.astype('float32')

# Scaling the range of the image to [-1, 1]
# Because we are using tanh as the activation function in the last layer of the generator
# and tanh restricts the weights in the range [-1, 1]
X_train = (X_train - 127.5) / 127.5

X_train.shape

generator = Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2)))
generator.add(BatchNormalization())
generator.add(Reshape((7,7,128)))
generator.add(Conv2DTranspose(64, kernel_size=5,strides=2,padding='same', activation=LeakyReLU(0.2)))
generator.add(BatchNormalization())
generator.add(Conv2DTranspose(1, kernel_size=5,strides=2, padding="same", activation="tanh"))

print('generator_model')
generator.summary()

discriminator = Sequential()
discriminator.add(Convolution2D(64, kernel_size=5,strides=2, input_shape=(28,28,1), padding="same", activation=LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Convolution2D(128, kernel_size=5,strides=2, padding="same", activation=LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))

print('discriminator_model')
discriminator.summary()

generator.compile(loss='binary_crossentropy', optimizer=Adam(),metrics=['accuracy'])
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(),metrics=['accuracy'])

discriminator.trainable = False
ganInput = Input(shape=(100,))
# getting the output of the generator
# and then feeding it to the discriminator
# new model = D(G(input))
x = generator(ganInput)
ganOutput = discriminator(x)
gan = Model(input=ganInput, output=ganOutput)
gan.compile(loss='binary_crossentropy', optimizer=Adam(),metrics=['accuracy'])
print('GAN_model')
gan.summary()


def train(epoch=10, batch_size=128):
    
    D_loss=np.zeros((epoch,1))
    D_acc=np.zeros((epoch,1))
    G_loss=np.zeros((epoch,1))
    G_acc=np.zeros((epoch,1))
    x=np.arange(0,epoch)
    
    for i in range(epoch):
      
      if math.floor(i/50)==(i/50):
        print('epoch={}'.format(i))
      
      for j in range(1):  
        # Input for the generator
        noise_input = np.random.rand(batch_size, 100)

        # getting random images from X_train of size=batch_size 
              # these are the real images that will be fed to the discriminator
        image_batch = X_train[np.random.randint(0, X_train.shape[0], size=batch_size)]

              # these are the predicted images from the generator
        predictions = generator.predict(noise_input, batch_size=batch_size)

              # the discriminator takes in the real images and the generated images
        X = np.concatenate([predictions, image_batch])

              # labels for the discriminator
        y_discriminator = [0]*batch_size + [1]*batch_size

              # Let's train the discriminator
        discriminator.trainable = True
        D=discriminator.fit(X, y_discriminator,verbose=0)
      
      for k in range(3):
              # Let's train the generator
        noise_input = np.random.rand(batch_size, 100)
        y_generator = [1]*batch_size
        discriminator.trainable = False
        G=gan.fit(noise_input, y_generator,verbose=0)
      
      D_loss[i]= D.history['loss']
      G_loss[i]= G.history['loss']
      D_acc[i]= D.history['acc']
      G_acc[i]= G.history['acc']

    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(x,D_acc)
    axs[0].plot(x,G_acc)
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].legend(['discriminator', 'generator'], loc='best')
    # summarize history for loss
    axs[1].plot(x,D_loss)
    axs[1].plot(x,G_loss)
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].legend(['discriminator', 'generator'], loc='best')
    plt.show()  

train(3000, 128)

def plot_output():
    try_input = np.random.rand(100, 100)
    preds = generator.predict(try_input)

    plt.figure(figsize=(10,10))
    for i in range(preds.shape[0]):
        plt.subplot(10, 10, i+1)
        plt.imshow(preds[i, :, :, 0], cmap='gray')
        plt.axis('off')
    
    # tight_layout minimizes the overlap between 2 sub-plots
    plt.tight_layout()
  
plot_output()
   